In [76]:
import pandas as pd
import numpy as np
import sqlite3
pd.options.mode.chained_assignment = None

Чтобы унифицировать названия всех регионов во всех таблицах и в geojson, создадим следующий словарь:

In [77]:
region_names = {'Сумма': 'Вся Россия',
                'ЦИК России': 'Вся Россия',
                'Камчатская область': 'Камчатский край',
                'Пермская область': 'Пермский край',
                'Читинская область': 'Забайкальский край',
                'город Москва': 'Город Москва',
                'город Санкт-Петербург': 'Город Санкт-Петербург',
                'город Севастополь': 'Город Севастополь',
                'Агинский Бурятский автономный округ': 'Забайкальский край',
                'Коми-Пермяцкий автономный округ': 'Пермский край',
                'Корякский автономный округ': 'Камчатский край',
                'Таймырский (Долгано-Ненецкий) автономный округ': 'Красноярский край',
                'Усть-Ордынский Бурятский автономный округ': 'Иркутская область',
                'Эвенкийский автономный округ': 'Красноярский край',
                'Чувашская Республика': 'Чувашская Республика - Чувашия',
                'Ханты-Мансийский автономный округ': 'Ханты-Мансийский автономный округ - Югра'}

#Президентские выборы

Из таблицы выборов с записями в виде регионов и столбцами в виде большого количества подробностей о явке создадим две таблицы: со статистикой по каждому кандидату в каждом регионе и со статистикой по явке в каждом регионе.

In [78]:
p_elections_2004_alldata = pd.read_csv('Результаты_выборов_президента_2004.csv', sep=';')
p_elections_2004_alldata.head()

,district_name,"Число избирателей, внесенных в список",Число полученных избирательных бюллетеней,"Число избирательных бюллетеней, выданных досрочно","Число избирательных бюллетеней, выданных в день голосования","Число избирательных бюллетеней, выданных вне помещения",Число погашенных избирательных бюллетеней,Число избирательных бюллетеней в переносных ящиках,Число бюллетеней в стационарных ящиках для голосования,Число недействительных избирательных бюллетеней,...,"Число открепительных удостоверений, выданных в ТИК",Число утраченных избирательных бюллетеней,Число не учтенных при получении избирательных бюллетеней,Глазьев Сергей Юрьевич,Малышкин Олег Александрович,Миронов Сергей Михайлович,Путин Владимир Владимирович,Хакамада Ирина Муцуовна,Харитонов Николай Михайлович,Против всех
0,Сумма,108064281,105744103,129022,64831236,4621172,36162189,4749595,64754683,578847,...,40227,643,159,2850330,1405396,524392,49563020,2671519,9514224,2396550
1,Республика Адыгея (Адыгея),321051,314778,0,226053,17927,70798,17926,225862,1747,...,70,0,0,7319,3593,1220,184326,3896,35224,6463
2,Республика Алтай,140046,135953,914,79358,7390,48291,8304,79332,674,...,98,0,0,2974,1213,643,65751,3074,11812,1495
3,Республика Башкортостан,2894559,2872563,0,2462023,116395,294118,116386,2461145,12699,...,614,120,93,28603,12748,8420,2365768,28748,102189,18356
4,Республика Бурятия,672225,665607,2681,408801,36441,217684,39121,408640,5448,...,72,0,0,12812,7234,3461,298120,32299,76483,11904


In [79]:
p_elections_2004_alldata['district_name'] = p_elections_2004_alldata['district_name'].map(
    lambda x: region_names[x.strip()] if x.strip() in region_names else x.strip())
p_elections_2004_alldata = p_elections_2004_alldata.groupby('district_name', as_index=False).sum()

p_elections_2004_region = p_elections_2004_alldata.iloc[:, [0, 1]]
p_elections_2004_region.columns = ['region', 'voters']
p_elections_2004_region['came'] = p_elections_2004_alldata.iloc[:, 3] + p_elections_2004_alldata.iloc[:,
                                                                        4] + p_elections_2004_alldata.iloc[:, 5]
p_elections_2004_region['voted'] = p_elections_2004_alldata.iloc[:, 9] + p_elections_2004_alldata.iloc[:, 10]
p_elections_2004_region['year'] = 2004

p_elections_2004_candidate = pd.DataFrame(
    {'region': [], 'candidate': [], 'votes': np.array([]).astype(int), 'year': np.array([]).astype(int)})
for column in p_elections_2004_alldata.columns[-7:]:
    p_elections_2004_candidate = pd.concat([p_elections_2004_candidate, pd.DataFrame(
        {'region': p_elections_2004_alldata['district_name'], 'candidate': [column] * len(p_elections_2004_alldata),
         'votes': p_elections_2004_alldata[column], 'year': [2004] * len(p_elections_2004_alldata)})])

Аналогично для 2008.

In [80]:
p_elections_2008_alldata = pd.read_csv('Результаты_выборов_президента_2008.csv', sep=';')
p_elections_2008_alldata['district_name'] = p_elections_2008_alldata['district_name'].map(
    lambda x: region_names[x.strip()] if x.strip() in region_names else x.strip())
p_elections_2008_alldata = p_elections_2008_alldata.groupby('district_name', as_index=False).sum()

p_elections_2008_region = p_elections_2008_alldata.iloc[:, [0, 1]]
p_elections_2008_region.columns = ['region', 'voters']
p_elections_2008_region['came'] = p_elections_2008_alldata.iloc[:, 3] + p_elections_2008_alldata.iloc[:,
                                                                        4] + p_elections_2008_alldata.iloc[:, 5]
p_elections_2008_region['voted'] = p_elections_2008_alldata.iloc[:, 9] + p_elections_2008_alldata.iloc[:, 10]
p_elections_2008_region['year'] = 2008

p_elections_2008_candidate = pd.DataFrame(
    {'region': [], 'candidate': [], 'votes': np.array([]).astype(int), 'year': np.array([]).astype(int)})
for column in p_elections_2008_alldata.columns[-4:]:
    p_elections_2008_candidate = pd.concat([p_elections_2008_candidate, pd.DataFrame(
        {'region': p_elections_2008_alldata['district_name'], 'candidate': [column] * len(p_elections_2008_alldata),
         'votes': p_elections_2008_alldata[column], 'year': [2008] * len(p_elections_2008_alldata)})])

Аналогично для 2012.

In [81]:
p_elections_2012_alldata = pd.read_csv('Результаты_выборов_президента_2012.csv', sep=';')
p_elections_2012_alldata['district name'] = p_elections_2012_alldata['district name'].map(
    lambda x: region_names[x.strip()] if x.strip() in region_names else x.strip())
p_elections_2012_alldata = p_elections_2012_alldata.groupby('district name', as_index=False).sum()

p_elections_2012_region = p_elections_2012_alldata.iloc[:, [0, 2]]
p_elections_2012_region.columns = ['region', 'voters']
p_elections_2012_region['came'] = p_elections_2012_alldata.iloc[:, 4] + p_elections_2012_alldata.iloc[:,
                                                                        5] + p_elections_2012_alldata.iloc[:, 6]
p_elections_2012_region['voted'] = p_elections_2012_alldata.iloc[:, 10] + p_elections_2012_alldata.iloc[:, 11]
p_elections_2012_region['year'] = 2012

p_elections_2012_candidate = pd.DataFrame(
    {'region': [], 'candidate': [], 'votes': np.array([]).astype(int), 'year': np.array([]).astype(int)})
for column in p_elections_2012_alldata.columns[-5:]:
    p_elections_2012_candidate = pd.concat([p_elections_2012_candidate, pd.DataFrame(
        {'region': p_elections_2012_alldata['district name'], 'candidate': [column] * len(p_elections_2012_alldata),
         'votes': p_elections_2012_alldata[column], 'year': [2012] * len(p_elections_2012_alldata)})])

Аналогично для 2018.

In [82]:
p_elections_2018_alldata = pd.read_csv('Результаты_выборов_президента_2018.csv', sep=';')
p_elections_2018_alldata['district name'] = p_elections_2018_alldata['district name'].map(
    lambda x: region_names[x.strip()] if x.strip() in region_names else x.strip())
p_elections_2018_alldata = p_elections_2018_alldata.groupby('district name', as_index=False).sum()

p_elections_2018_region = p_elections_2018_alldata.iloc[:, [0, 2]]
p_elections_2018_region.columns = ['region', 'voters']
p_elections_2018_region['came'] = p_elections_2018_alldata.iloc[:, 4] + p_elections_2018_alldata.iloc[:,
                                                                        5] + p_elections_2018_alldata.iloc[:, 6]
p_elections_2018_region['voted'] = p_elections_2018_alldata.iloc[:, 10] + p_elections_2018_alldata.iloc[:, 11]
p_elections_2018_region['year'] = 2018

p_elections_2018_candidate = pd.DataFrame(
    {'region': [], 'candidate': [], 'votes': np.array([]).astype(int), 'year': np.array([]).astype(int)})
for column in p_elections_2018_alldata.columns[-8:]:
    p_elections_2018_candidate = pd.concat([p_elections_2018_candidate, pd.DataFrame(
        {'region': p_elections_2018_alldata['district name'], 'candidate': [column] * len(p_elections_2018_alldata),
         'votes': p_elections_2018_alldata[column], 'year': [2018] * len(p_elections_2018_alldata)})])

Соединим получившиеся таблицы в две большие таблицы с информацией о всех доступных нам президентских выборах.

In [83]:
p_elections_candidate = pd.concat(
    [p_elections_2004_candidate, p_elections_2008_candidate, p_elections_2012_candidate, p_elections_2018_candidate])
p_elections_region = pd.concat(
    [p_elections_2004_region, p_elections_2008_region, p_elections_2012_region, p_elections_2018_region])

#Выборы в Госдуму

Чтобы унифицировать названия всех партий, которые почему-то на каждых выборах записываются немного иначе, создадим следующий словарь:

In [84]:
parties = {
    '1. "ЕДИНЕНИЕ"': 'Единение',
    '2. "СОЮЗ ПРАВЫХ СИЛ"': 'Союз правых сил',
    '3. "РОССИЙСКАЯ ПАРТИЯ ПЕНСИОНЕРОВ И ПАРТИЯ СОЦИАЛЬНОЙ СПРАВЕДЛИВОСТИ"': 'Блок "Российская партия пенсионеров и Партия социальной справедливости"',
    '4. "Российская демократическая партия "ЯБЛОКО"': 'Яблоко',
    '5. "За Русь Святую"': 'За Русь Святую',
    '6. "Объединенная Российская партия "Русь"': 'Русь',
    '7. "Новый курс - Автомобильная Россия"': 'Блок "Новый курс - Автомобильная Россия"',
    '8. "Народно-республиканская партия России"': 'Народно-республиканская партия России',
    '9. "Российская экологическая партия "Зеленые"': 'Зеленые',
    '10. "Аграрная партия России"': 'Аграрная партия России',
    '11. "Истинные патриоты России"': 'Истинные патриоты России',
    '12. "НАРОДНАЯ ПАРТИЯ Российской Федерации"': 'Народная партия России',
    '13. "Демократическая партия России"': 'Демократическая партия России',
    '14. "Великая Россия - Евразийский Союз"': 'Блок "Великая Россия - Евразийский Союз"',
    '15. "Партия СЛОН"': 'Партия СЛОН',
    '16. "Родина" (народно-патриотический союз)"': 'Блок "Родина"',
    '17. "Партия Мира и Единства (ПМЕ)"': 'Партия Мира и Единства',
    '18. "ЛДПР"': 'ЛДПР',
    '19. "Партия Возрождения России - Российская партия ЖИЗНИ"': 'Блок "Партия возрождения России - Российская партия жизни"',
    '20. "Политическая партия "Единая Россия"': 'Единая Россия',
    '21. "Российская Конституционно-демократическая партия"': 'Российская Конституционно-демократическая партия',
    '22. "Развитие предпринимательства"': 'Развитие предпринимательства',
    '23. "Коммунистическая партия Российской Федерации (КПРФ)"': 'КПРФ',
    'Против всех': 'Против всех',
    '1.Политическая партия «Аграрная партия России»': 'Аграрная партия России',
    '2.Всероссийская политическая партия «Гражданская Сила»': 'Гражданская Сила',
    '3.Политическая партия «Демократическая партия России»': 'Демократическая партия России',
    '4.Политическая партия «Коммунистическая партия Российской Федерации»': 'КПРФ',
    '5.Политическая партия «СОЮЗ ПРАВЫХ СИЛ»': 'Союз правых сил',
    '6.Политическая партия «Партия социальной справедливости»': 'Партия социальной справедливости',
    '7.Политическая партия «Либерально-демократическая партия России»': 'ЛДПР',
    '8.Политическая партия «СПРАВЕДЛИВАЯ РОССИЯ: РОДИНА/ПЕНСИОНЕРЫ/ЖИЗНЬ»': 'Справедливая Россия',
    '9.Политическая партия «ПАТРИОТЫ РОССИИ»': 'Патриоты России',
    '10.Всероссийская политическая партия «ЕДИНАЯ РОССИЯ»': 'Единая Россия',
    '11.Политическая партия «Российская объединенная демократическая партия «ЯБЛОКО»': 'Яблоко',
    '1. Политическая партия СПРАВЕДЛИВАЯ РОССИЯ': 'Справедливая Россия',
    '2. Политическая партия "Либерально-демократическая партия России"': 'ЛДПР',
    '3. Политическая партия "ПАТРИОТЫ РОССИИ" ': 'Патриоты России',
    '4. Политическая партия "Коммунистическая партия Российской Федерации"': 'КПРФ',
    '5. Политическая партия "Российская объединенная демократическая партия "ЯБЛОКО"': 'Яблоко',
    '6. Всероссийская политическая партия "ЕДИНАЯ РОССИЯ"': 'Единая Россия',
    '7. Всероссийская политическая партия "ПРАВОЕ ДЕЛО"': 'Правое дело',
    }

Так как внутри некоторых регионов есть отдельные избирательные округа со своими названиями, идущими после названия региона, напишем небольшую функцию, которая будет оставлять только название региона.

In [85]:
def region_name_edit(region):
    for i in range(len(region) - 2):
        if region[i + 1] == ' ' and region[i + 2] == '-':
            return region[:i + 1]
    return region

Проделаем все аналогичные манипуляции и с выборами в Госдуму

In [86]:
gd_elections_2003_alldata = pd.read_csv('Результаты_выборов_депутатов_по_фед_2003.csv', sep=';')
gd_elections_2003_alldata['district_name'] = gd_elections_2003_alldata['district_name'].map(lambda x: region_name_edit(x))
gd_elections_2003_alldata['district_name'] = gd_elections_2003_alldata['district_name'].map(
    lambda x: region_names[x.strip()] if x.strip() in region_names else x.strip())
gd_elections_2003_alldata = gd_elections_2003_alldata.groupby('district_name', as_index=False).sum()

gd_elections_2003_region = gd_elections_2003_alldata.iloc[:, [0, 1]]
gd_elections_2003_region.columns = ['region', 'voters']
gd_elections_2003_region['came'] = gd_elections_2003_alldata.iloc[:, 3] + gd_elections_2003_alldata.iloc[:,
                                                                        4] + gd_elections_2003_alldata.iloc[:, 5]
gd_elections_2003_region['voted'] = gd_elections_2003_alldata.iloc[:, 9] + gd_elections_2003_alldata.iloc[:, 10]
gd_elections_2003_region['year'] = 2003

gd_elections_2003_candidate = pd.DataFrame(
    {'region': [], 'candidate': [], 'votes': np.array([]).astype(int), 'year': np.array([]).astype(int)})
for column in gd_elections_2003_alldata.columns[-24:]:
    gd_elections_2003_candidate = pd.concat([gd_elections_2003_candidate, pd.DataFrame(
        {'region': gd_elections_2003_alldata['district_name'], 'candidate': [parties[column]] * len(gd_elections_2003_alldata),
         'votes': gd_elections_2003_alldata[column], 'year': [2003] * len(gd_elections_2003_alldata)})])

In [87]:
gd_elections_2007_alldata = pd.read_csv('Результаты_выборов_депутатов_по_фед_2007.csv', sep=';')
gd_elections_2007_alldata['district_name'] = gd_elections_2007_alldata['district_name'].map(lambda x: region_name_edit(x))
gd_elections_2007_alldata['district_name'] = gd_elections_2007_alldata['district_name'].map(
    lambda x: region_names[x.strip()] if x.strip() in region_names else x.strip())
gd_elections_2007_alldata = gd_elections_2007_alldata.groupby('district_name', as_index=False).sum()

gd_elections_2007_region = gd_elections_2007_alldata.iloc[:, [0, 1]]
gd_elections_2007_region.columns = ['region', 'voters']
gd_elections_2007_region['came'] = gd_elections_2007_alldata.iloc[:, 3] + gd_elections_2007_alldata.iloc[:,
                                                                        4] + gd_elections_2007_alldata.iloc[:, 5]
gd_elections_2007_region['voted'] = gd_elections_2007_alldata.iloc[:, 9] + gd_elections_2007_alldata.iloc[:, 10]
gd_elections_2007_region['year'] = 2007

gd_elections_2007_candidate = pd.DataFrame(
    {'region': [], 'candidate': [], 'votes': np.array([]).astype(int), 'year': np.array([]).astype(int)})
for column in gd_elections_2007_alldata.columns[-11:]:
    gd_elections_2007_candidate = pd.concat([gd_elections_2007_candidate, pd.DataFrame(
        {'region': gd_elections_2007_alldata['district_name'], 'candidate': [parties[column]] * len(gd_elections_2007_alldata),
         'votes': gd_elections_2007_alldata[column], 'year': [2007] * len(gd_elections_2007_alldata)})])

In [88]:
gd_elections_2011_alldata = pd.read_csv('Результаты выборов в думу 2011.csv', sep=';')
gd_elections_2011_alldata['name'] = gd_elections_2011_alldata['name'].map(lambda x: region_name_edit(x))
gd_elections_2011_alldata['name'] = gd_elections_2011_alldata['name'].map(
    lambda x: region_names[x.strip()] if x.strip() in region_names else x.strip())
gd_elections_2011_alldata = gd_elections_2011_alldata.groupby('name', as_index=False).sum()

gd_elections_2011_region = gd_elections_2011_alldata.iloc[:, [0, 1]]
gd_elections_2011_region.columns = ['region', 'voters']
gd_elections_2011_region['came'] = gd_elections_2011_alldata.iloc[:, 3] + gd_elections_2011_alldata.iloc[:,
                                                                        4] + gd_elections_2011_alldata.iloc[:, 5]
gd_elections_2011_region['voted'] = gd_elections_2011_alldata.iloc[:, 9] + gd_elections_2011_alldata.iloc[:, 10]
gd_elections_2011_region['year'] = 2011

gd_elections_2011_candidate = pd.DataFrame(
    {'region': [], 'candidate': [], 'votes': np.array([]).astype(int), 'year': np.array([]).astype(int)})
for column in gd_elections_2011_alldata.columns[-7:]:
    gd_elections_2011_candidate = pd.concat([gd_elections_2011_candidate, pd.DataFrame(
        {'region': gd_elections_2011_alldata['name'], 'candidate': [parties[column]] * len(gd_elections_2011_alldata),
         'votes': gd_elections_2011_alldata[column], 'year': [2011] * len(gd_elections_2011_alldata)})])

Соединим получившиеся таблицы в две большие таблицы с информацией о всех доступных нам депутатских выборах.

In [89]:
gd_elections_candidate = pd.concat(
    [gd_elections_2003_candidate, gd_elections_2007_candidate, gd_elections_2011_candidate])
gd_elections_region = pd.concat(
    [gd_elections_2003_region, gd_elections_2007_region, gd_elections_2011_region])

#Создание базы данных

Создадим базу данных elections.db

In [90]:
connection = sqlite3.connect("elections.db")
cursor = connection.cursor()

Добавим в БД таблицу с названиями регионов и их id.

In [91]:
cursor.execute("""
CREATE TABLE region(region_id INTEGER PRIMARY KEY AUTOINCREMENT, region_name VARCHAR(100))
""")
for r in pd.concat([p_elections_region, gd_elections_region])['region'].unique():
    cursor.execute(f"""
    INSERT INTO region(region_name)
    VALUES ('{r}')
    """)

Добавим в БД таблицу с именами кандидатов в президенты и их id.

In [92]:
cursor.execute("""
CREATE TABLE p_candidate(candidate_id INTEGER PRIMARY KEY AUTOINCREMENT, candidate_name VARCHAR(100))
""")
for c in p_elections_candidate['candidate'].unique():
    cursor.execute(f"""
    INSERT INTO p_candidate(candidate_name)
    VALUES ('{c}')
    """)

Добавим в БД таблицу с названиями партий-кандидатов в Госдуму и их id.

In [93]:
cursor.execute("""
CREATE TABLE gd_candidate(candidate_id INTEGER PRIMARY KEY AUTOINCREMENT, candidate_name VARCHAR(100))
""")
for c in gd_elections_candidate['candidate'].unique():
    cursor.execute(f"""
    INSERT INTO gd_candidate(candidate_name)
    VALUES ('{c}')
    """)

Добавим в БД таблицу с результатами всех кандидатов в президенты на всех выборах во всех регионах.

In [94]:
cursor.execute("""
CREATE TABLE president_elections_cand(id INTEGER PRIMARY KEY AUTOINCREMENT, region_id INT, candidate_id INT, votes INT, year INT, FOREIGN KEY (region_id)  REFERENCES region (region_id) ON DELETE CASCADE, FOREIGN KEY (candidate_id)  REFERENCES p_candidate (candidate_id) ON DELETE CASCADE)
""")
for i in range(len(p_elections_candidate)):
    cursor.execute(f"""
    INSERT INTO president_elections_cand(region_id, candidate_id, votes, year)
    VALUES ((SELECT region_id FROM region WHERE region_name = '{p_elections_candidate.iloc[i]['region']}'), (SELECT candidate_id FROM p_candidate WHERE candidate_name = '{p_elections_candidate.iloc[i]['candidate']}'), {p_elections_candidate.iloc[i]['votes']}, {p_elections_candidate.iloc[i]['year']})
    """)

Добавим в БД таблицу с результатами всех партий-кандидатов в Госдуму на всех выборах во всех регионах.

In [95]:
cursor.execute("""
CREATE TABLE gd_elections_cand(id INTEGER PRIMARY KEY AUTOINCREMENT, region_id INT, candidate_id INT, votes INT, year INT, FOREIGN KEY (region_id)  REFERENCES region (region_id) ON DELETE CASCADE, FOREIGN KEY (candidate_id)  REFERENCES gd_candidate (candidate_id) ON DELETE CASCADE)
""")
for i in range(len(gd_elections_candidate)):
    cursor.execute(f"""
    INSERT INTO gd_elections_cand(region_id, candidate_id, votes, year)
    VALUES ((SELECT region_id FROM region WHERE region_name = '{gd_elections_candidate.iloc[i]['region']}'), (SELECT candidate_id FROM gd_candidate WHERE candidate_name = '{gd_elections_candidate.iloc[i]['candidate']}'), {gd_elections_candidate.iloc[i]['votes']}, {gd_elections_candidate.iloc[i]['year']})
    """)

Добавим в БД таблицу с информацией о явке на президентских выборах во всех регионах за все годы.

In [96]:
cursor.execute("""
CREATE TABLE president_elections_region(id INTEGER PRIMARY KEY, region_id INT, voters INT, came INT, voted INT, year INT, FOREIGN KEY (region_id)  REFERENCES region (region_id) ON DELETE CASCADE)
""")
for i in range(len(p_elections_region)):
    cursor.execute(f"""
    INSERT INTO president_elections_region(region_id, voters, came, voted, year)
    VALUES ((SELECT region_id FROM region WHERE region_name = '{p_elections_region.iloc[i]['region']}'), {p_elections_region.iloc[i]['voters']},  {p_elections_region.iloc[i]['came']},  {p_elections_region.iloc[i]['voted']},  {p_elections_region.iloc[i]['year']})
    """)

Добавим в БД таблицу с информацией о явке на выборах в Госдуму во всех регионах за все годы.

In [97]:
cursor.execute("""
CREATE TABLE gd_elections_region(id INTEGER PRIMARY KEY, region_id INT, voters INT, came INT, voted INT, year INT, FOREIGN KEY (region_id)  REFERENCES region (region_id) ON DELETE CASCADE)
""")
for i in range(len(gd_elections_region)):
    cursor.execute(f"""
    INSERT INTO gd_elections_region(region_id, voters, came, voted, year)
    VALUES ((SELECT region_id FROM region WHERE region_name = '{gd_elections_region.iloc[i]['region']}'), {gd_elections_region.iloc[i]['voters']},  {gd_elections_region.iloc[i]['came']},  {gd_elections_region.iloc[i]['voted']},  {gd_elections_region.iloc[i]['year']})
    """)

Закоммитим изменения.

In [98]:
connection.commit()

#JSON для карты России

In [99]:
import json
import pandas as pd
import plotly as plt
import plotly.express as px
from urllib.request import urlopen

Скачаем geojson с картой регионов России, после чего изменим названия регионов в соответствии с их названиями в нашей БД и присвоим им внутри geojson id соответствующие region_id из таблицы region.

In [100]:
with urlopen('https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/russia.geojson') as response:
    counties = json.load(response)

regions_republic_1 = ['Бурятия', 'Тыва', 'Адыгея', 'Татарстан', 'Марий Эл',
                      'Чувашия', 'Северная Осетия - Алания', 'Алтай',
                      'Дагестан', 'Ингушетия', 'Башкортостан']
regions_republic_2 = ['Удмуртская республика', 'Кабардино-Балкарская республика',
                      'Карачаево-Черкесская республика', 'Чеченская республика']
regions_republic_3 = ['Республика Адыгея', 'Республика Татарстан']
cities = ['Москва', 'Санкт-Петербург']

for k in range(len(counties['features'])):
    if counties['features'][k]['properties']['name'] in regions_republic_1:
        counties['features'][k]['properties']['name'] = 'Республика ' + counties['features'][k]['properties']['name']
    if counties['features'][k]['properties']['name'] in regions_republic_2:
        counties['features'][k]['properties']['name'] = counties['features'][k]['properties']['name'].title()
    if counties['features'][k]['properties']['name'] in regions_republic_3:
        counties['features'][k]['properties']['name'] = counties['features'][k]['properties']['name'] + f" ({counties['features'][k]['properties']['name'].split()[1]})"
    if counties['features'][k]['properties']['name'] in cities:
        counties['features'][k]['properties']['name'] = 'Город ' + counties['features'][k]['properties']['name']
    if counties['features'][k]['properties']['name'] == 'Республика Чувашия':
        counties['features'][k]['properties']['name'] = 'Чувашская Республика - Чувашия'
    if counties['features'][k]['properties']['name'] in region_names:
        counties['features'][k]['properties']['name'] = region_names[counties['features'][k]['properties']['name']]

for k in range(len(counties['features'])):
    counties['features'][k]['id'] = int(pd.read_sql_query(f"SELECT region_id FROM region WHERE region_name = '{counties['features'][k]['properties']['name']}'", connection)['region_id'][0])

with open('russia.geojson', 'w', encoding='utf-8') as f:
    json.dump(counties, f)